In [1]:
from __future__ import (
    division,
    absolute_import,
    with_statement,
    print_function,
    unicode_literals,
)
import os
import tqdm
import cv2
import torch
import argparse
import torch.nn as nn
import numpy as np
import pickle as pkl
from common import Config, ConfigRandLA
from models.ffb6d import FFB6D
from datasets.ycb.ycb_dataset import Dataset as YCB_Dataset
from datasets.linemod.linemod_dataset import Dataset as LM_Dataset
from utils.pvn3d_eval_utils_kpls import cal_frame_poses, cal_frame_poses_lm
from utils.basic_utils import Basic_Utils
try:
    from neupeak.utils.webcv2 import imshow, waitKey
except ImportError:
    from cv2 import imshow, waitKey

/home/xs2445/envs/ffb6dEnv/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
AXIS_PTS = np.array([[0,0,0],
                     [0.1,0,0],
                     [0,0.1,0],
                     [0,0,0.1]])

COLORS = [[250,0,0],
          [0,250,0],
          [0,0,250]]

def ensure_fd(fd):
    if not os.path.exists(fd):
        os.system('mkdir -p {}'.format(fd))

def load_checkpoint(model=None, optimizer=None, filename="checkpoint"):
    filename = "{}.pth.tar".format(filename)

    assert os.path.isfile(filename), "==> Checkpoint '{}' not found".format(filename)
    print("==> Loading from checkpoint '{}'".format(filename))
    try:
        checkpoint = torch.load(filename)
    except Exception:
        checkpoint = pkl.load(open(filename, "rb"))
    epoch = checkpoint.get("epoch", 0)
    it = checkpoint.get("it", 0.0)
    best_prec = checkpoint.get("best_prec", None)
    if model is not None and checkpoint["model_state"] is not None:
        ck_st = checkpoint['model_state']
        if 'module' in list(ck_st.keys())[0]:
            tmp_ck_st = {}
            for k, v in ck_st.items():
                tmp_ck_st[k.replace("module.", "")] = v
            ck_st = tmp_ck_st
        model.load_state_dict(ck_st)
    if optimizer is not None and checkpoint["optimizer_state"] is not None:
        optimizer.load_state_dict(checkpoint["optimizer_state"])
    print("==> Done")
    return it, epoch, best_prec

In [3]:
config = Config(ds_name='linemod', cls_type='ape')

bs_utils = Basic_Utils(config)

checkpoint_path='/home/xs2445/6895/FFB6D/ffb6d/train_log/linemod/checkpoints/ape/FFB6D_ape_best.pth.tar'

In [4]:
rndla_cfg = ConfigRandLA
model = FFB6D(
    n_classes=config.n_objects, n_pts=config.n_sample_points, rndla_cfg=rndla_cfg,
    n_kps=config.n_keypoints
)
# model.cuda()
load_checkpoint(
    model, None, filename=checkpoint_path[:-8]
)


loading resnet34 pretrained mdl.
==> Loading from checkpoint '/home/xs2445/6895/FFB6D/ffb6d/train_log/linemod/checkpoints/ape/FFB6D_ape_best.pth.tar'
==> Done


(0.0, 0, None)

In [5]:
print(model)

FFB6D(
  (cnn_pre_stages): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (rndla_pre_stages): Conv1d(
    (conv): Conv1d(9, 8, kernel_size=(1,), stride=(1,), bias=False)
    (bn): BatchNorm1d(
      (bn): BatchNorm1d(8, eps=1e-06, momentum=0.99, affine=True, track_running_stats=True)
    )
    (activation): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (cnn_ds_stages): ModuleList(
    (0): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bi

In [22]:
test_ds = LM_Dataset('test', cls_type='ape')
print(config.mini_batch_size)
test_loader = torch.utils.data.DataLoader(
        test_ds, batch_size=1, shuffle=False,
        num_workers=20
    )

cls_id in lm_dataset.py 1
test_dataset_size:  1050
3


In [23]:
count = 0
for b in test_loader:
    if count>0:
        break
    count += 1
    print(b)

kps_pth in get_kps: datasets/linemod/kps_orb9_fps/ape_8_kps.txt
kps_pth in get_kps:kps_pth in get_kps:  kps_pth in get_kps:datasets/linemod/kps_orb9_fps/ape_8_kps.txtdatasets/linemod/kps_orb9_fps/ape_8_kps.txtkps_pth in get_kps: 
kps_pth in get_kps:
datasets/linemod/kps_orb9_fps/ape_8_kps.txt
 datasets/linemod/kps_orb9_fps/ape_8_kps.txt 
datasets/linemod/kps_orb9_fps/ape_8_kps.txtkps_pth in get_kps:kps_pth in get_kps:
kps_pth in get_kps:kps_pth in get_kps:  kps_pth in get_kps:datasets/linemod/kps_orb9_fps/ape_8_kps.txt   
datasets/linemod/kps_orb9_fps/ape_8_kps.txtdatasets/linemod/kps_orb9_fps/ape_8_kps.txtkps_pth in get_kps:datasets/linemod/kps_orb9_fps/ape_8_kps.txt

datasets/linemod/kps_orb9_fps/ape_8_kps.txtkps_pth in get_kps:kps_pth in get_kps: 

 datasets/linemod/kps_orb9_fps/ape_8_kps.txt datasets/linemod/kps_orb9_fps/ape_8_kps.txt
kps_pth in get_kps:kps_pth in get_kps:datasets/linemod/kps_orb9_fps/ape_8_kps.txt

  kps_pth in get_kps:kps_pth in get_kps:datasets/linemod/kps_orb9_

In [18]:
print(b.keys())
for key,val in b.items():
    b[key] = torch.tensor(val)

dict_keys(['rgb', 'cld_rgb_nrm', 'choose', 'labels', 'rgb_labels', 'dpt_map_m', 'RTs', 'kp_targ_ofst', 'ctr_targ_ofst', 'cls_ids', 'ctr_3ds', 'kp_3ds', 'cld_xyz0', 'cld_nei_idx0', 'cld_sub_idx0', 'cld_interp_idx0', 'r2p_ds_nei_idx0', 'p2r_ds_nei_idx0', 'cld_xyz1', 'cld_nei_idx1', 'cld_sub_idx1', 'cld_interp_idx1', 'r2p_ds_nei_idx1', 'p2r_ds_nei_idx1', 'cld_xyz2', 'cld_nei_idx2', 'cld_sub_idx2', 'cld_interp_idx2', 'r2p_ds_nei_idx2', 'p2r_ds_nei_idx2', 'cld_xyz3', 'cld_nei_idx3', 'cld_sub_idx3', 'cld_interp_idx3', 'r2p_ds_nei_idx3', 'p2r_ds_nei_idx3', 'r2p_up_nei_idx0', 'p2r_up_nei_idx0', 'r2p_up_nei_idx1', 'p2r_up_nei_idx1', 'r2p_up_nei_idx2', 'p2r_up_nei_idx2'])


/home/xs2445/envs/ffb6dEnv/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
a = test_ds.__getitem__(0)
# for key,val in a.items():
#     a[key] = torch.tensor(val).cuda()

print(a.keys())

kps_pth in get_kps: datasets/linemod/kps_orb9_fps/ape_8_kps.txt
dict_keys(['rgb', 'cld_rgb_nrm', 'choose', 'labels', 'rgb_labels', 'dpt_map_m', 'RTs', 'kp_targ_ofst', 'ctr_targ_ofst', 'cls_ids', 'ctr_3ds', 'kp_3ds', 'cld_xyz0', 'cld_nei_idx0', 'cld_sub_idx0', 'cld_interp_idx0', 'r2p_ds_nei_idx0', 'p2r_ds_nei_idx0', 'cld_xyz1', 'cld_nei_idx1', 'cld_sub_idx1', 'cld_interp_idx1', 'r2p_ds_nei_idx1', 'p2r_ds_nei_idx1', 'cld_xyz2', 'cld_nei_idx2', 'cld_sub_idx2', 'cld_interp_idx2', 'r2p_ds_nei_idx2', 'p2r_ds_nei_idx2', 'cld_xyz3', 'cld_nei_idx3', 'cld_sub_idx3', 'cld_interp_idx3', 'r2p_ds_nei_idx3', 'p2r_ds_nei_idx3', 'r2p_up_nei_idx0', 'p2r_up_nei_idx0', 'r2p_up_nei_idx1', 'p2r_up_nei_idx1', 'r2p_up_nei_idx2', 'p2r_up_nei_idx2'])


In [8]:
# print(config.mini_batch_size)
# test_loader = torch.utils.data.DataLoader(
#         test_ds, batch_size=config.test_mini_batch_size, shuffle=False,
#         num_workers=20
#     )
print(torch.__version__)

1.10.0+cu102


In [19]:
# model.cuda()
# traced_model = torch.jit.trace()
traced_model = torch.jit.script(model, example_inputs=b)

# traced_model
# torch.jit.save(traced_model, save_path)

RuntimeError: Can't redefine method: forward on class: __torch__.models.ffb6d.FFB6D (of Python compilation unit at: 0x5411c90)

In [27]:
model.cpu()

torch.onnx.export(model, (b, {}), 'model_onnx.pt')

RuntimeError: expected scalar type Byte but found Float